# 可迭代对象

可迭代对象，`Iterable`。

Python中任意的对象，只要它定义了可以返回一个迭代器的`__iter__`方法，或者定义了可以支持下标索引的`__getitem__`方法（这些双下划线方法会在其他章节中全面解释），那么它就是一个可迭代对象。

简单说，可迭代对象就是能提供迭代器的任意对象。

在Python中，可以使用`collections.abc`模块中的`Iterable`类来检查一个对象是否是可迭代的。

如果对象实现了`__iter__()`方法或者`__getitem__()`方法（并且能接受从零开始的索引），那么它就是可迭代的。

以下是一个如何判断对象是否可迭代的例子：

In [1]:
from collections.abc import Iterable


def is_iterable(obj):
    return isinstance(obj, Iterable)


print(is_iterable([1, 2, 3]))  # 列表是可迭代的，输出 True
print(is_iterable('abc'))  # 字符串是可迭代的，输出 True
print(is_iterable(123))  # 整数不是可迭代的，输出 False


True
True
False


如果不想或不能使用`collections.abc`模块，还可以通过检查对象是否有`__iter__`方法或者`__getitem__`方法来判断：

In [2]:
def is_iterable(obj):
    return hasattr(obj, '__iter__') or hasattr(obj, '__getitem__')

# 示例
print(is_iterable([1, 2, 3]))  # 输出 True
print(is_iterable('abc'))      # 输出 True
print(is_iterable(123))        # 输出 False

True
True
False


请注意，这些方法并不完美。例如，一个对象可能有一个`__getitem__`方法但并不真的支持整数索引迭代，

或者可能有一个自定义的`__iter__`方法但它不返回一个迭代器。

不过在大多数情况下，这种方式足够用来判断一个对象是否是可迭代的。

# 迭代器

在Python中，迭代器（Iterator）是遵守迭代器协议的对象，它包含两个基本的方法：`__iter__()`和`__next__()`。

- `__iter__()` 方法返回迭代器对象本身。这是使用`for`和`in`语句时所必需的。

- `__next__()` 方法返回容器的下一个元素。当没有更多元素时，应该抛出`StopIteration`异常。

任何支持迭代的对象都会实现这两个方法。

Python中的列表（list）、元组（tuple）、字典（dict）、集合（set）和字符串（str）都是可迭代的，

因为它们都实现了`__iter__()`方法以返回一个迭代器，该迭代器又实现了`__next__()`方法。

下面是如何实现一个简单的计数器作为迭代器的例子：

In [3]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1

# 使用迭代器
for i in Counter(3, 8):
    print(i)


3
4
5
6
7
8


在这个例子中，`Counter` 类实现了一个迭代器，它可以迭代给定范围内的所有整数。

当迭代到`high`属性所定义的边界之外时，它会抛出`StopIteration`异常，告诉`for`循环停止迭代。

此外，你也可以使用内置的函数next()来显式地从迭代器中获取下一个元素：

In [4]:
counter = Counter(3, 8)
print(next(counter)) # 输出 3
print(next(counter)) # 输出 4
# ...依此类推，直到 StopIteration 被抛出

3
4


当`StopIteration`异常被抛出时，迭代器将到达其边界，无法再提供更多的值。

这种机制使得迭代器非常适合表示无限大的数据流，因为它们不需要在开始迭代之前将所有元素都加载到内存中。

这次我们创建一个迭代器，这个迭代器将对给定的数据集合实施Fibonacci数列的规则，即下一个数是前两个数的和。

它会生成数列中的每个数，直到达到指定的序列长度。

In [5]:
class Fibonacci:
    def __init__(self, max_number):
        self.max_number = max_number
        self.first = 0
        self.second = 1
        self.count = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.count >= self.max_number:
            raise StopIteration

        if self.count == 0:
            self.count += 1
            return self.first
        elif self.count == 1:
            self.count += 1
            return self.second
        else:
            self.count += 1
            self.first, self.second = self.second, self.first + self.second
            return self.first

# 使用迭代器
fib = Fibonacci(5)
for num in fib:
    print(num)

0
1
1
1
2


在这个例子中，`Fibonacci` 类实现了一个迭代器，它可以产生Fibonacci数列。

这个迭代器将会在产生`max_number`个数之后停止。

这里`__iter__()`方法返回迭代器对象自己，而`__next__()`方法计算下一个Fibonacci数，并在需要时更新内部状态。

如果迭代器达到了最大数量，`__next__()`将抛出`StopIteration`异常，这将结束迭代。

# 生成器

在Python中，生成器是一种用于创建迭代器的简单直接的工具。

一个生成器是一个包含了`yield`表达式的函数，这种函数在你对其进行迭代时，每次`yield`一个值。

生成器在每次迭代中产生值（使用`yield`语句），并通过保持函数的状态来保持其局部变量和控制流的位置，

这与普通函数执行到最后或者遇到`return`语句就完全结束的行为是不同的。

使用生成器可以非常方便地实现相同的Fibonacci数列迭代器：

In [6]:
def fibonacci(max_number):
    a, b = 0, 1
    count = 0
    while count < max_number:
        yield a
        a, b = b, a + b
        count += 1

# 使用生成器
for num in fibonacci(5):
    print(num)

0
1
1
2
3


在这个`fibonacci`函数中，`yield`语句用来一次返回一个Fibonacci数。

每次迭代时，生成器从它上次`yield`出来的地方继续执行（维持其所有局部变量的状态），这直到下一个`yield`语句，或者当函数执行完毕，生成器自动停止迭代，这时候不再有`yield`，也就不会产生`StopIteration`异常。

生成器提供了一种更简洁的方式来实现简单的迭代器的功能，无需编写一个实现`__iter__()`和`__next__()`方法的类。

生成器的另一个好处是它们在内存使用上更高效，因为在任何给定时间内，只有迭代器当前的状态需要被存储在内存中，而不是整个数据集。
这在处理大数据流或可能无限的序列时特别有用。